# Notebook 2:  Data Cleaning 

In this notebook we merge our raw data tables from Notebook 1 and clean the salaries.
1.  **Merging**: We update our 2020 free agent data with the 2021 next-season salaries, and also merge current-season (2021-22) stats and salaries together.  This gives us two new dataframes:  df2020_merged and df2021_merged.
2.  **Cleaning salaries**: We convert salaries into numerical form and bucket them into "minimum salary", "maximum salary" and "in-between" bins.  We also normalize salaries from all years to the 2021 salary cap for consistency.

## Imports

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import random

from bs4 import BeautifulSoup, Comment
import requests
import lxml
import unicodedata

import json, pickle

## Utility Functions

In [2]:
def strip_accents_and_punctuation(text):
    '''Normalize player name spellings'''
    try:
        text = unicode(text, 'utf-8')
    except NameError: # unicode is a default on python 3 
        pass
    text = unicodedata.normalize('NFD', text)\
           .encode('ascii', 'ignore')\
           .decode("utf-8")
    return str(text).replace('.','').replace(',','').replace("'",'')

# (1) Merge Scraped Data

In [3]:
df2020 = pd.read_csv('./data/df2020_raw.csv')
df2021salaries = pd.read_csv('./data/dfcurrentsalaries.csv')
df2021stats = pd.read_csv('./data/dfcurrentstats.csv')
df2021urls = pd.read_csv('./data/dfplayer_to_url.csv')

## a. Update df2020 with 2021 salaries

In [4]:
df2020_merged = pd.merge(df2020, df2021salaries, left_on="Name", right_on="Name", how="left")
df2020_merged = df2020_merged.drop(columns=['NextSal', 'Unnamed: 0', 'CurrentTeam'])
df2020_merged = df2020_merged.rename(columns = {'CurrentSalary':'NextSal'})
df2020_merged = df2020_merged.set_index('NameYear')
df2020_merged.head(3)

,Name,Pos,Type,OTm,PrevStats,NTm,ID,Weight,Height,Age,...,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,PrevSal,PrevYear,NextSal
NameYear,,,,,,,,,,,,,,,,,,,,,
Chris_Paul_2020,Chris Paul,G,UFA,PHO,"16.4 Pts, 4.5 Reb, 8.9 Ast",PHO,/players/p/paulch01.html,175.0,6-0,35.0,...,2.7,9.2,0.201,3.5,1.1,4.6,3.6,"$41,358,814",2020,"$30,800,000"
Kawhi_Leonard_2020,Kawhi Leonard,F,UFA,LAC,"24.8 Pts, 6.5 Reb, 5.2 Ast",LAC,/players/l/leonaka01.html,225.0,6-7,29.0,...,2.4,8.8,0.238,6.0,1.3,7.3,4.2,"$34,379,100",2020,"$39,344,900"
Enes_Freedom_2020,Enes Freedom,C,UFA,POR,"11.2 Pts, 11.0 Reb, 1.2 Ast",BOS,/players/k/kanteen01.html,250.0,6-10,28.0,...,1.7,7.5,0.204,2.1,-1.4,0.6,1.2,"$5,005,350",2020,"$2,641,691"


## b. Update df2021 with 2021 salaries and player urls

In [5]:
df2021_merged = pd.merge(df2021stats, df2021salaries, left_on="Player", right_on="Name", how="left")
df2021_merged = pd.merge(df2021_merged, df2021urls, left_on="Player", right_on="Name", how="left")

df2021_merged = df2021_merged.drop(columns=['Name_x', 'Name_y','Unnamed: 0','Unnamed: 0_x','Unnamed: 0_y','Rk'])
df2021_merged = df2021_merged.rename(columns = {'Player':'Name'})

df2021_merged.head(3)

,Name,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,TOV%,TRB%,TS%,USG%,VORP,WS,WS/48,CurrentSalary,CurrentTeam,ID
0,Precious Achiuwa,C,22,TOR,73,28,23.6,3.6,8.3,.439,...,11.3,14.9,.503,18.5,-0.2,2.5,.070,"$2,711,280",TOR,/players/a/achiupr01.html
1,Steven Adams,C,28,MEM,76,75,26.3,2.8,5.1,.547,...,19.6,19.9,.560,12.0,2.0,6.8,.163,"$17,073,171",MEM,/players/a/adamsst01.html
2,Bam Adebayo,C,24,MIA,56,56,32.6,7.3,13.0,.557,...,14.4,17.5,.608,25.0,2.7,7.2,.188,"$28,103,550",MIA,/players/a/adebaba01.html


## c. Update df2021 with current team and total stats

This is a little bit tricky because some players played for multiple teams.  For example, NAW switched teams mid-season and has three rows:

* Name    /   Team
* Nickeil Alexander-Walker  /    TOT
* Nickeil Alexander-Walker  /    NOP
* Nickeil Alexander-Walker  /    UTA

We want to keep his **season stats** from the first row (TOT) but also indicate his **current team** from the last row (UTA).  The same pattern holds for all players who played for multiple teams.





In [6]:
# create current team name filter 
justteam = ~df2021_merged.duplicated(subset=['Name'],keep='last')
# create total stats filter
juststats = ~df2021_merged.duplicated(subset=['Name'],keep='first')


df2021_merged = (pd.merge(df2021_merged[juststats], df2021_merged[justteam][['Name','Tm']], 
                          left_on="Name", right_on="Name", how="left")
                            .rename(columns ={'Tm_x':'TeamOrTot', 'Tm_y':'Team'}) 
         )

df2021_merged['NameYear'] = df2021_merged['Name'].apply(strip_accents_and_punctuation).str.replace(' ','_') + '_2021' 
df2021_merged = df2021_merged.set_index('NameYear')

Minor bookkeeping:
Update a few Teams with correct info (only applies to 15 players, not that important):


In [7]:
mask = df2021_merged.CurrentTeam.notna() & (df2021_merged.CurrentTeam != df2021_merged.Team)
df2021_merged.loc[mask,'Team'] = df2021_merged.loc[mask,'CurrentTeam']

df2021_merged = df2021_merged.drop(columns=["CurrentTeam"])
df2021_merged = df2021_merged.rename(columns= {'CurrentSalary': 'Salary'})

df2021_merged.head(5)

,Name,Pos,Age,TeamOrTot,G,GS,MP,FG,FGA,FG%,...,TOV%,TRB%,TS%,USG%,VORP,WS,WS/48,Salary,ID,Team
NameYear,,,,,,,,,,,,,,,,,,,,,
Precious_Achiuwa_2021,Precious Achiuwa,C,22,TOR,73,28,23.6,3.6,8.3,.439,...,11.3,14.9,.503,18.5,-0.2,2.5,.070,"$2,711,280",/players/a/achiupr01.html,TOR
Steven_Adams_2021,Steven Adams,C,28,MEM,76,75,26.3,2.8,5.1,.547,...,19.6,19.9,.560,12.0,2.0,6.8,.163,"$17,073,171",/players/a/adamsst01.html,MEM
Bam_Adebayo_2021,Bam Adebayo,C,24,MIA,56,56,32.6,7.3,13.0,.557,...,14.4,17.5,.608,25.0,2.7,7.2,.188,"$28,103,550",/players/a/adebaba01.html,MIA
Santi_Aldama_2021,Santi Aldama,PF,21,MEM,32,0,11.3,1.7,4.1,.402,...,9.9,12.6,.452,18.4,-0.3,0.3,.044,"$1,994,520",/players/a/aldamsa01.html,MEM
LaMarcus_Aldridge_2021,LaMarcus Aldridge,C,36,BRK,47,12,22.3,5.4,9.7,.550,...,8.0,13.4,.604,22.4,0.7,3.1,.141,"$2,641,691",/players/a/aldrila01.html,BRK


## Save merged data

In [8]:
df2020_merged.to_csv(r'/Users/andrei/Dropbox/Metis/HoopsHero/data/df2020_merged.csv')
df2021_merged.to_csv(r'/Users/andrei/Dropbox/Metis/HoopsHero/data/df2021_merged.csv')

# (2) Clean salary data

## Load files

In [9]:
df2016_raw = pd.read_csv('./data/df2016_raw.csv')
df2017_raw = pd.read_csv('./data/df2017_raw.csv')
df2018_raw = pd.read_csv('./data/df2018_raw.csv')
df2019_raw = pd.read_csv('./data/df2019_raw.csv')
df2020_merged = pd.read_csv('./data/df2020_merged.csv')

df  = pd.concat([df2016_raw, df2017_raw, df2018_raw, df2019_raw, df2020_merged])
df = df.set_index('NameYear')

df2021 = pd.read_csv('./data/df2021_merged.csv')
df2021 = df2021.set_index('NameYear')
df2021 = df2021.drop("Player_2021")  #spurious entry

## Delete unnecessary entries

We first employ a very light common-sense filter on our entries.  To train our player value model, we wish to keep only players who: 
1.  Played in at least one game the previous year
2.  Had a salary the next year (i.e. received a contract in free agency)

In [10]:
#  1. Mask for players who played in at least one game the previous year

df['G'] = df['G'].apply(lambda s: pd.to_numeric(s, errors='coerce'))
mask1 = df['G'].isna()
df = df[~mask1]

#  2. Mask for players who had a salary the next year

mask2  = (df['NextSal'] == '0') | df['NextSal'].isna()  
df = df[~mask2]
df.sample(3)

,Name,Pos,Type,OTm,PrevStats,NTm,ID,Weight,Height,Age,...,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,PrevSal,NextSal,PrevYear
NameYear,,,,,,,,,,,,,,,,,,,,,
ETwaun_Moore_2020,E'Twaun Moore,G-F,UFA,PHO,"4.9 Pts, 1.7 Reb, 1.5 Ast",ORL,/players/m/mooreet01.html,191.0,6-3,31.0,...,0.4,0.4,0.049,-3.1,0.2,-2.9,-0.1,"$2,331,593","$2,641,691",2020
Wesley_Matthews_2020,Wesley Matthews,G,UFA,LAL,"4.8 Pts, 1.6 Reb, 0.9 Ast",MIL,/players/m/matthwe02.html,220.0,6-4,34.0,...,1.5,1.7,0.073,-3.1,1.4,-1.7,0.1,"$3,623,000","$1,958,501",2020
Quincy_Acy_2017,Quincy Acy,F-C,UFA,BRK,"5.9 Pts, 3.7 Reb, 0.8 Ast",NaN,/players/a/acyqu01.html,240.0,6-7,27.0,...,1.1,1.0,0.036,-2.6,0.1,-2.5,-0.2,"$1,709,538","$213,948",2017


### Important year-by-year data:  salary cap, minimum contract salary, and maximum contract salary.  We use this to clean and normalize salary data.

In [11]:
# Salary Cap Data taken from spotrac.com/nba/cba

# salary cap by season
salarycap = {2016:   94.143,  2017: 99.093,   2018: 101.869,   2019:  109.14,   
             2020:   109.14,  2021: 112.414}   

# we are tracking the "maximum min salary", for players with 10+ years of experience
minsalary = {2016:   1.552,   2017: 2.329,   2018: 2.375,  2019:  2.565,   
             2020:   2.565,   2021: 2.642}

# we are tracking the "minimum max salary", for players with 0-6 years of experience
maxsalary = {2016:  22.117,   2017:  24.773,   2018: 25.467,  2019: 27.285,   
             2020:  27.285,   2021:  28.103}

# minsalarynorm = {year: (salary*salarycap[2021]/salarycap[year]) for year, salary in minsalary.items()}
#     1.85M for 2016, 2.642 the other years
# maxsalarynorm = {year: (salary*salarycap[2021]/salarycap[year]) for year, salary in maxsalary.items()}
#     26.4M for 2016, 28.103 the other years


# from wikipedia.org/wiki/List_of_National_Basketball_Association_seasons
# note that the 2019 season teams actually played 63-75 games

totalgames: {2016: 82,    2017: 82,    2018: 82,   2019: 75,   
             2020: 72,    2021: 82}


In [12]:
def clean_salary(sal, year):
    ''' Adds all salaries earned per year into one salary (in millions of dollars)
        "< $Minimum" is treated as $0
    '''
    newlist = [0]
    for elem in sal.split('$'):
        try:
            elem = elem.replace('<','').replace(',','').replace('Minimum','0')
            elem =  float(elem)
            newlist.append(elem)
        except:
            continue
            
    salary = sum(newlist)/1000000
    
    return salary


def norm_salary(sal, year):
    ''' Adds all salaries earned per year into one salary (in millions of dollars)
        "< $Minimum" is treated as $0
        Then normalizes by salary cap (in 2021 salary cap dollars)
        Buckets salaries into 'Min', 'Max' and in-between
    '''
    newlist = [0]
    for elem in sal.split('$'):
        try:
            elem = elem.replace('<','').replace(',','').replace('Minimum','0')
            elem =  float(elem)
            newlist.append(elem)
        except:
            continue
            
    salary = sum(newlist)/1000000
    salary = salary*salarycap[2021]/salarycap[year]
    
    if salary <= minsalary[2021]:   #2.642M
        return 'Min'
    if salary >= maxsalary[2021]:   #28.103M
        return 'Max'
    
    return salary

for index, row in df.iterrows():
    
    prevyear = df.loc[index, 'PrevYear']
    nextyear = df.loc[index, 'PrevYear'] + 1 

    prevsal = df.loc[index, 'PrevSal']
    nextsal = df.loc[index, 'NextSal']
    
    df.loc[index, 'PrevSalClean'] = clean_salary(prevsal, prevyear)
    df.loc[index, 'NextSalClean'] = clean_salary(nextsal, nextyear)
    df.loc[index, 'PrevSalNorm'] = norm_salary(prevsal, prevyear)
    df.loc[index, 'NextSalNorm'] = norm_salary(nextsal, nextyear)


# test = df.loc['Troy_Williams_2017'].PrevSal
# print(test)
# print(clean_salary(test))

      
# df['PrevSalClean'] = df['PrevSal'].apply(clean_salary)
# df['NextSalClean'] = df['NextSal'].apply(clean_salary)

cols2 = ['NextSal', 'NextSalClean','NextSalNorm']
df[cols2].sample(8)

,NextSal,NextSalClean,NextSalNorm
NameYear,,,
Udonis_Haslem_2016,"$1,471,382",1.471382,Min
Lance_Thomas_2018,"$262,152",0.262152,Min
Antonio_Blakeney_2018,"$1,488,231",1.488231,Min
Reggie_Bullock_2020,"$9,536,000",9.536000,9.536
CJ_Watson_2016,"$333,334",0.333334,Min
JaMychal_Green_2019,"$7,199,760",7.199760,7.41574
Willie_Cauley-Stein_2019,"$4,100,000",4.100000,4.222992
Kostas_Antetokounmpo_2019,< $Minimum,0.000000,Min


In [13]:
for index, row in df2021.iterrows():
    
    salary = df2021.loc[index, 'Salary']
    try:
        df2021.loc[index, 'SalClean'] = clean_salary(salary, 2021)
    except:
        df2021.loc[index, 'SalClean'] = np.nan
        
    try:
        df2021.loc[index, 'SalNorm'] = norm_salary(salary, 2021)
    except:
        df2021.loc[index, 'SalNorm'] = np.nan


df2021.sample(3)

,Name,Pos,Age,TeamOrTot,G,GS,MP,FG,FGA,FG%,...,TS%,USG%,VORP,WS,WS/48,Salary,ID,Team,SalClean,SalNorm
NameYear,,,,,,,,,,,,,,,,,,,,,
Omer_Yurtseven_2021,Omer Yurtseven,C,23,MIA,56,12,12.6,2.3,4.4,.526,...,.546,19.9,0.2,2.1,.145,"$1,489,065",/players/y/yurtsom01.html,MIA,1.489065,Min
Joe_Johnson_2021,Joe Johnson,SG,40,BOS,1,0,2.0,1.0,1.0,1.000,...,1.000,22.0,0.0,0.0,.335,NaN,/players/j/johnsjo02.html,BOS,NaN,NaN
Sam_Hauser_2021,Sam Hauser,SF,24,BOS,26,0,6.1,0.9,1.9,.460,...,.650,14.5,0.2,0.6,.171,"$313,737",/players/h/hausesa01.html,BOS,0.313737,Min


## Save cleaned data

In [14]:
df.to_csv(r'/Users/andrei/Dropbox/Metis/HoopsHero/data/cleaned_pastyears.csv')
df2021.to_csv(r'/Users/andrei/Dropbox/Metis/HoopsHero/data/cleaned_currentyear.csv')